*This is Baseline*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/2. Pubblicazioni/3. In Corso/2. Identifying Fake Reviews for Refund Purposes/4. comparative evaluation among PARROT and state-of-the-art models/Baseline/"

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
from sklearn.model_selection import train_test_split

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

#### Inference

In [ ]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
from transformers import AutoTokenizer, GPT2Model
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

# LLM Data


In [ ]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
import torch
import os
import sys
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report,f1_score

In [62]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")
model = torch.load(f'{path}models/detector-base.pt')

In [56]:
def predict(query, model, tokenizer, device="cpu"):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.model_max_length - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()
    return real

In [57]:
from torch import cuda
device = 'cpu'

In [58]:
def metrics(df, llm, preds_probas,preds):
    y_true = df.label.values
    y_pred = preds

    # Create the evaluation report.
    evaluation_report = classification_report(y_true, y_pred, labels=[0,1], target_names=["Real","Fake"])

    confusion_matrix(y_true,y_pred)
    acc = accuracy_score(y_true,y_pred)
    precision = precision_score(y_true,y_pred,labels=["Real","Fake"])
    recall = recall_score(y_true,y_pred,labels=["Real","Fake"])
    f1_s = f1_score(y_true,y_pred,labels=["Real","Fake"])

    print(f"\nReport: {evaluation_report};\nAccuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}; f1_score:{f1_s}")

    with open(f"Result/{llm}/gpt2_predictions_{llm}.txt", "w") as f:
        f.write(f"{evaluation_report}\nMetrics:\nAccuracy_score: {acc}\nPrecision_score: {precision}\nRecall_score: {recall}\nF1_Score: {f1_s}")
    preds_df_rows = []

    for i, row in df.iterrows():
        query = row["content"]
        pred_prob = preds_probas[i]
        pred_label = preds[i]
        preds_df_rows.append([row["ID"], row["label"], pred_prob, pred_label])

    preds_df = pd.DataFrame(preds_df_rows, columns=["ID","Original_Label","Finetune_GPT2_Model_Probability","Finetune_GPT2_Model_Prediction"])
    preds_df.to_csv(f"Result//{llm}/GPT2_predictions_{llm}.csv", index=None)

In [59]:
def answer(df,llm):

  ammount = df.shape[0]

  ID = ""
  Title = ""
  Cont =""
  label = ""
  count_row= 1
  start = 0
  preds, preds_probas = [],[]
  for i,r in df.iterrows():
    # try:
      if i>= start:
            ID = r["ID"]
            Title = r["Title"]
            Cont = r["content"]
            label = r["label"]

            query = r["content"]
            pred = predict(query,model,tokenizer)
            preds_probas.append(pred)
            if pred >= 0.5:
                preds.append(1)
            else:
                preds.append(0)

            sys.stdout.write(
            "\r "+llm+" Done %i/%i                                " % (count_row, ammount))
            sys.stdout.flush()
            count_row = count_row + 1
    # except Exception as e:
    #     print(f"Restart: Index:{i}\nID:{ID},\nTitle:{Title},\nContenuto:{Cont},\nLabel:{label}")
    #     print(f"Index for restart: {i}\n")
    #     print(e)

  metrics(df=df, llm=llm,preds_probas= preds_probas,preds=preds)

In [60]:
import random

def shuffle(df):
    num_rows = df.shape[0]

    # Generate random indices
    random_indices = random.sample(range(num_rows), num_rows)

    # Shuffle dataframe rows based on random indices
    df = df.loc[random_indices]

    return df

In [61]:
LLMs = ["BARD", "CLAUDE", "GPT", "LAMA"]


for llm in LLMs:
    df = shuffle(pd.read_csv(f"/content/drive/MyDrive/2. Pubblicazioni/3. In Corso/2. Identifying Fake Reviews for Refund Purposes/4. comparative evaluation among PARROT and state-of-the-art models/Dataset/dataset_{llm}_recensioni_generate_eng.csv", sep=';', encoding = "ISO-8859-1"))
    answer(df, llm)
    print(f"\n\n{llm} DONE\n\n")

TypeError: ignored